# Logistic Regression Notebook
[Return to project overview](final_project_overview.ipynb)

### Andrew Larimer, Deepak Nagaraj, Daniel Olmstead, Michael Winton (W207-4-Summer 2018 Final Project)

In [8]:
# import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from util import our_train_test_split

# set default options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

%matplotlib inline

In [30]:
from functools import partial
from sklearn.model_selection import train_test_split
TEST_SIZE = 0.2
RANDOM_STATE = 207
PASSNYC_LABELS = [0,1]
our_train_test_split = partial(train_test_split,
                               stratify=y,
                               test_size=TEST_SIZE,
                               random_state=RANDOM_STATE)
merged_df = pd.read_csv('data_merged/combined_data_2018-07-18.csv')
X = merged_df.loc[:, ~merged_df.columns.isin(['high_registrations'])]
y = merged_df.loc[:, merged_df.columns.isin(['high_registrations'])]

train_data, test_data, train_labels, test_labels = our_train_test_split(X, y)
train_data

,dbn,num_shsat_test_takers,offers_per_student,pct_test_takers,school_name,district,zip,community_school,economic_need_index,school_income_estimate,percent_ell,percent_asian,percent_black,percent_hispanic,percent_black__hispanic,percent_white,student_attendance_rate,percent_of_students_chronically_absent,rigorous_instruction_percent,rigorous_instruction_rating,collaborative_teachers_percent,collaborative_teachers_rating,supportive_environment_percent,supportive_environment_rating,effective_school_leadership_percent,effective_school_leadership_rating,strong_family_community_ties_percent,strong_family_community_ties_rating,trust_percent,trust_rating,student_achievement_rating,average_ela_proficiency,average_math_proficiency,grade_7_ela_all_students_tested,grade_7_ela_4s_all_students,grade_7_ela_4s_american_indian_or_alaska_native,grade_7_ela_4s_black_or_african_american,grade_7_ela_4s_hispanic_or_latino,grade_7_ela_4s_asian_or_pacific_islander,grade_7_ela_4s_white,grade_7_ela_4s_multiracial,grade_7_ela_4s_limited_english_proficient,grade_7_ela_4s_economically_disadvantaged,grade_7_math_all_students_tested,grade_7_math_4s_all_students,grade_7_math_4s_american_indian_or_alaska_native,grade_7_math_4s_black_or_african_american,grade_7_math_4s_hispanic_or_latino,grade_7_math_4s_asian_or_pacific_islander,grade_7_math_4s_white,grade_7_math_4s_multiracial,grade_7_math_4s_limited_english_proficient,grade_7_math_4s_economically_disadvantaged,number_of_students_english,number_of_students_math,number_of_students_science,number_of_students_social_studies,number_of_classes_english,number_of_classes_math,number_of_classes_science,number_of_classes_social_studies,average_class_size_english,average_class_size_math,average_class_size_science,average_class_size_social_studies,school_pupil_teacher_ratio
421,22K014,41,0,19,J.H.S. 014 SHELL BANK,22,11235,0,0.708,NaN,18,11,48,17,64,24,92.0,29.0,86.0,3.0,91.0,3.0,80.0,3.0,86.0,3.0,71.0,2.0,92.0,3.0,3.0,2.32,2.41,165,8,0,3,1,0,2,0,0,7,178,14,0,2,2,0,6,0,1,14,767.0,606.0,765.0,622.0,37.0,29.0,36.0,28.0,20.729730,20.896552,21.250000,22.214286,12.5
344,32K554,22,0,17,ALL CITY LEADERSHIP SECONDARY SCHOOL,32,11237,0,0.667,NaN,2,6,11,77,89,5,97.0,8.0,92.0,4.0,99.0,4.0,96.0,4.0,92.0,4.0,83.0,3.0,98.0,4.0,3.0,3.28,3.47,60,22,0,2,18,0,0,0,0,20,60,19,0,2,15,0,0,0,0,17,330.0,180.0,180.0,180.0,13.0,6.0,6.0,6.0,25.384615,30.000000,30.000000,30.000000,23.9
279,19K654,15,0,16,VAN SICLEN COMMUNITY MIDDLE SCHOOL,19,11207,0,0.780,NaN,14,2,68,28,96,2,90.0,33.0,86.0,3.0,96.0,3.0,83.0,3.0,93.0,4.0,80.0,3.0,95.0,4.0,2.0,1.96,1.95,110,0,0,0,0,0,0,0,0,0,113,0,0,0,0,0,0,0,0,0,312.0,327.0,327.0,297.0,14.0,14.0,12.0,12.0,22.285714,23.357143,27.250000,24.750000,16.7
244,17K394,12,0,21,M.S. K394,17,11213,0,0.797,32470.47,4,6,77,15,92,1,92.0,24.0,89.0,3.0,86.0,3.0,83.0,2.0,80.0,3.0,82.0,2.0,85.0,2.0,2.0,2.35,2.21,57,1,0,1,0,0,0,0,0,0,57,0,0,0,0,0,0,0,0,0,284.0,308.0,154.0,154.0,13.0,14.0,7.0,7.0,21.846154,22.000000,22.000000,22.000000,14.0
379,08X562,29,0,30,BLUEPRINT MIDDLE SCHOOL,8,10472,0,0.796,NaN,15,23,18,56,74,1,92.0,29.0,93.0,3.0,95.0,3.0,80.0,2.0,93.0,4.0,79.0,3.0,91.0,3.0,2.0,2.23,2.17,102,2,0,0,0,1,0,0,0,1,101,4,0,0,0,4,0,0,0,4,49.0,331.0,383.0,NaN,2.0,14.0,16.0,NaN,24.500000,23.642857,23.937500,NaN,13.3
347,19K218,23,0,16,J.H.S. 218 JAMES P. SINNOTT,19,11208,0,0.857,NaN,22,20,36,43,79,1,89.0,41.0,87.0,2.0,85.0,2.0,82.0,2.0,80.0,3.0,79.0,3.0,89.0,3.0,3.0,2.16,2.14,123,4,0,0,1,0,0,0,0,2,127,11,0,0,1,0,0,0,1,7,776.0,417.0,424.0,387.0,35.0,19.0,19.0,18.0,22.171429,21.947368,22.315789,21.500000,14.6
252,28Q008,12,0,12,J.H.S. 008 RICHARD S. GROSSLEY,28,11433,1,0.694,NaN,13,12,55,21,77,1,90.0,36.0,88.0,2.0,90.0,2.0,83.0,2.0,74.0,2.0,75.0,2.0,87.0,3.0,3.0,2.21,2.14,117,1,1,0,0,0,0,0,0,1,119,4,0,0,1,3,0,0,0,4,347.0,377.0,377.0,345.0,15.0,15.0,16.0,15.0,23.133333,25.133333,23.562500,23.000000,12.6
444,29Q059,56,0,28,I.S. 059 SPRINGFIELD GARDENS,29,11413,0,0.468,NaN,2,1,91,5,96,1,93.0,20.0,94.0,3.0,87.0,3.0,81.0,3.0,79.0,2.0,78.0,2.0

In [29]:
from sklearn.linear_model import LogisticRegression
c_values = {'C': [0.010, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 1.000, 2.000, 3.000, 4.000]}
for c in c_values['C']:
    log = LogisticRegression(C=c, penalty='l2', random_state=207)
    log.fit(train_data, train_labels)
    log_predicted_labels = log.predict(test_data)
    log_score = metrics.f1_score(test_labels, log_predicted_labels, average='weighted')
    print("F1 score for C={}: {:.4f}".format(c, log_score))


ValueError: could not convert string to float: "YOUNG WOMEN'S LEADERSHIP SCHOOL OF THE BRONX"